In [26]:
import matplotlib.pyplot as plt
%matplotlib inline

In [27]:
import torch

In [28]:
import numpy as np
import torchvision as tv
import pandas as pd
import numpy as np
import time

In [29]:
BATCH_SIZE=256

train_dataset = tv.datasets.FashionMNIST('.', train=True, transform=tv.transforms.ToTensor(), download=True)
test_dataset = tv.datasets.FashionMNIST('.', train=False, transform=tv.transforms.ToTensor(), download=True)
train = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [30]:

train_dataset[0][0].shape

torch.Size([1, 28, 28])

In [31]:
'''model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 10)
)'''

'model = torch.nn.Sequential(\n    torch.nn.Flatten(),\n    torch.nn.Linear(784, 256),\n    torch.nn.ReLU(),\n    torch.nn.Linear(256, 10)\n)'

In [32]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 1024),
    torch.nn.BatchNorm1d(1024),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.1),
    torch.nn.Linear(1024, 512),
    torch.nn.BatchNorm1d(512),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(512, 256),
    torch.nn.BatchNorm1d(256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(256, 10)
)

In [33]:
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=1024, bias=True)
  (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): ReLU()
  (4): Dropout(p=0.1, inplace=False)
  (5): Linear(in_features=1024, out_features=512, bias=True)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): ReLU()
  (8): Dropout(p=0.2, inplace=False)
  (9): Linear(in_features=512, out_features=256, bias=True)
  (10): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): ReLU()
  (12): Dropout(p=0.3, inplace=False)
  (13): Linear(in_features=256, out_features=10, bias=True)
)

In [34]:
loss = torch.nn.CrossEntropyLoss()
trainer = torch.optim.Adam(model.parameters(), lr=.001)
num_epochs = 10

In [35]:
#from prettytable import PrettyTable
#x = PrettyTable()

In [36]:
#x.field_names = ["EPOCH", "TimeTaked", "train_loss", "train_acc", "test_loss", "test_acc"]
def train_model():
    for ep in range(num_epochs):
        train_iters, train_passed  = 0, 0
        train_loss, train_acc = 0., 0.
        start=time.time()
        
        model.train()
        for X, y in train:
            trainer.zero_grad()
            y_pred = model(X)
            l = loss(y_pred, y)
            l.backward()
            trainer.step()
            train_loss += l.item()
            train_acc += (y_pred.argmax(dim=1) == y).sum().item()
            train_iters += 1
            train_passed += len(X)
        
        test_iters, test_passed  = 0, 0
        test_loss, test_acc = 0., 0.
        model.eval()
        for X, y in test:
            y_pred = model(X)
            l = loss(y_pred, y)
            test_loss += l.item()
            test_acc += (y_pred.argmax(dim=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)
            
        print("epoch: {}, taked_time: {:.3f}, train_loss: {:.3f}, train_acc: {:.3f}, test_loss: {:.3f}, test_acc: {:.3f}".format(
            ep, time.time() - start, train_loss / train_iters, train_acc / train_passed,
            test_loss / test_iters, test_acc / test_passed)
        )
        #x.add_row([round(ep,3),round(time.time() - start,3), round(train_loss / train_iters,3), round(train_acc / train_passed,3), round(test_loss / test_iters,3), round(test_acc / test_passed,3)])
        #print(x.get_string())

In [37]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [38]:
train_model()

epoch: 0, taked_time: 46.503, train_loss: 0.465, train_acc: 0.836, test_loss: 0.413, test_acc: 0.845
epoch: 1, taked_time: 42.830, train_loss: 0.341, train_acc: 0.875, test_loss: 0.382, test_acc: 0.857
epoch: 2, taked_time: 42.193, train_loss: 0.304, train_acc: 0.889, test_loss: 0.362, test_acc: 0.864
epoch: 3, taked_time: 43.030, train_loss: 0.276, train_acc: 0.898, test_loss: 0.353, test_acc: 0.869
epoch: 4, taked_time: 39.994, train_loss: 0.257, train_acc: 0.905, test_loss: 0.335, test_acc: 0.878
epoch: 5, taked_time: 40.277, train_loss: 0.240, train_acc: 0.911, test_loss: 0.333, test_acc: 0.880
epoch: 6, taked_time: 40.410, train_loss: 0.226, train_acc: 0.915, test_loss: 0.320, test_acc: 0.886
epoch: 7, taked_time: 40.432, train_loss: 0.210, train_acc: 0.922, test_loss: 0.329, test_acc: 0.883
epoch: 8, taked_time: 41.122, train_loss: 0.198, train_acc: 0.926, test_loss: 0.326, test_acc: 0.887
epoch: 9, taked_time: 36.636, train_loss: 0.190, train_acc: 0.929, test_loss: 0.343, test_a